In [0]:
df = spark.read.format("csv").option("header", True).load("dbfs:/FileStore/datasets/authors_publications.csv")
df.show(5)

In [0]:
from pyspark.sql import functions as F

# Example: Filter rows where 'publication_year' >= 2020 and aggregate count by 'author'
filtered_df = df.filter(F.col("Year") >= 2020)
agg_df = filtered_df.groupBy("Authors").agg(F.count("*").alias("publication_count"))
display(agg_df)